In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "Colab Notebooks/airbnb_scraper"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)


Your working directory was changed to /content/drive/My Drive/Colab Notebooks/airbnb_scraper

An empty text file was created there. You can also run !pwd to confirm the current working directory.


Install the dependencies

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

In [ ]:
from bs4 import BeautifulSoup
import webbrowser
import requests
import pandas as pd
import cv2 
import urllib.request as urli
import os 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
import pickle

The options have to be adjusted to make everything work in colab and drive. chromedrive has to be saved in google drive in the right folder

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options= options)

The list was cut in two in order too not go over the time limit of 24 hours of colab

In [ ]:
listings = pd.read_csv("listings.csv")
prefix = "https://airbnb.de"
suffix = "/photos"
urls = listings["listing_url"][12722:]
ids = listings["id"][12722:]


In [ ]:
id_dict = dict()
for id, url in tqdm(zip(ids, urls), total = len(ids)):    
    id_list = []
    try:
        driver = webdriver.Chrome('chromedriver',options= options)
        
        driver.get(url+suffix)

        body = driver.find_element_by_tag_name("body")
        for i in range(10): # scrolls java_script page down. If Pictures are missing, increase
            body.send_keys(Keys.TAB)
            body.send_keys(Keys.PAGE_DOWN)


        html = driver.page_source
        html_content = BeautifulSoup(html, "html.parser")
        gallery_items = html_content.findAll("img", {"class": "_6tbg2q"})
    except:
        continue
    for i, item in enumerate(gallery_items[5:]):
        try:
            img_link = item.get("src")
            img_title = item.get("alt")
            path_name = "/content/drive/MyDrive/Colab Notebooks/airbnb_scraper/pictures/pictures_2st_half_8750_to_full/"+str(id)+"_"+str(i)+".png" #adjust to location where pic should saved
            if os.path.exists(path_name):
                id_list.append(img_title)
                continue
            else:
                urli.urlretrieve(img_link, path_name)
                id_list.append(img_title)
            
        except:
            continue

    id_dict[id] = id_list
meta_images = open("/content/drive/MyDrive/Colab Notebooks/airbnb_scraper/pictures/pictures_2st_half_8750_to_full/data.pkl", "wb") #adjust to location where pic should saved
pickle.dump(id_dict, meta_images)
meta_images.close

In [ ]:
listings = pd.read_csv("listings.csv")

ids = listings["id"]



38128708